# WEEK 4 

## Question 1

In [62]:
import networkx as nx
import time
g = nx.DiGraph()
filename = "c17.net"
with open(filename) as f:
    for line in f:
        parts = line.strip().split()
        gate_id, gate_type, *inputs, output = parts
        g.add_node(output, gate_type=gate_type)
        for input_node in inputs:
            g.add_edge(input_node, output)
try:
    topological_order = list(nx.topological_sort(g))
    print(topological_order)
except:
    print("the circuit can't be evaluated")

['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']


### Explanation:


The code then creates an empty directed graph, represented as a DiGraph object, using g = nx.DiGraph()<br>
Then opens the file<br>
For each line, the code extracts the gate ID, gate type, input nodes, and output node<br>
then the edges and nodes are added to the circuit to determine to flow<br>
then the topological_sort to sort in topological order

## Question 2

First, I have written a function to evaluate the gates based on the type of gate

In [63]:
def evaluate_gate(gate_type, input_states_list):
    if gate_type == "and2":
        return int(all(input_states_list))
    elif gate_type == "nand2":
        return int(not all(input_states_list))
    elif gate_type == "or2":
        return int(any(input_states_list))
    elif gate_type == "nor2":
        return int(not any(input_states_list))
    elif gate_type == "xor2":
        return int(sum(input_states_list) % 2 == 1)
    elif gate_type == "xnor2":
        return int(sum(input_states_list) % 2 == 0)
    elif gate_type == "inv":
        return int(not input_states_list)
    elif gate_type == "buf":
        return int(input_states_list[0])
    else:
        raise ValueError("Invalid gate type")

### Evaluation using Topological order

In [64]:

count=1
inputs = []
with open("c17.inputs") as f:
    
        inputs=f.readlines()
        l=len(inputs)-1
        prim=inputs[0].split()

for j in range (1,l+1):
    input_vector=inputs[j].split()
    states = {}
    for i, input_id in enumerate(prim):
        if g.in_degree(input_id) == 0:
            states[input_id] = int(input_vector[i])
    for gate_id in topological_order:
        if g.in_degree(gate_id) > 0:
            input_gate_id=list(g.predecessors(gate_id))
            input_state_list=[states[n] for n in input_gate_id]
            gate = g.nodes[gate_id]['gate_type']
            states[gate_id] = evaluate_gate(gate, input_state_list)
    print(f"List of states for input vector {count}")
    for node_id in sorted(g.nodes):
        print(states[node_id],end=" ")
    count+=1
    print("\n")


List of states for input vector 1
0 1 1 1 0 0 0 1 1 1 0 

List of states for input vector 2
0 0 0 0 1 0 0 1 1 1 1 

List of states for input vector 3
1 0 0 0 0 0 0 1 1 1 1 

List of states for input vector 4
0 0 0 0 1 1 1 1 0 1 1 

List of states for input vector 5
1 1 1 0 1 1 1 0 0 1 1 

List of states for input vector 6
1 1 1 1 1 0 0 0 1 1 0 

List of states for input vector 7
1 1 1 0 1 1 0 0 0 1 1 

List of states for input vector 8
1 1 1 1 0 0 0 1 1 1 0 

List of states for input vector 9
0 1 1 1 1 0 1 1 1 0 0 

List of states for input vector 10
0 0 0 0 1 1 0 1 0 1 1 



the code first reads the circuit
<br>the code initializes a dictionary to store the input and output states of the circuit. It then iterates through the gates of the circuit in topological order, computes the output state of each gate based on its input states using the evaluate_gate() function, and updates the dictionary with the new output state.

### Evaluation using Event driven Simulation

In [65]:

import networkx as nx
from queue import Queue
filename = "c17.net"
with open(filename) as f:
    for line in f:
        parts = line.strip().split()
        gate_id, gate_type, *inputs, output = parts
        g.add_node(output, gate_type=gate_type)
        for input_node in inputs:
            g.add_edge(input_node, output)
with open("c17.inputs") as f:
    inputs = f.readlines()
    l = len(inputs) - 1
    primary_inputs = inputs[0].split()
input_vector=inputs[1].split()    
queue = []
for node_id in g.nodes():
    queue.append(node_id)
states = {node_id: None for node_id in g.nodes()}
    
for node_id in sorted(g.nodes):
    print(f"{node_id}",end=" ")

print("\n")
    
for i, input_id in enumerate(primary_inputs):
    states[input_id] = int(input_vector[i])
while queue:
    node_id = queue.pop(0)
    if(states[node_id]==1 or states[node_id]==0):
        continue        
    input_states_list = [states[input_gate_id] for input_gate_id in g.predecessors(node_id)]
    if None in input_states_list:
        queue.append(node_id)
    else:
        gate = g.nodes[node_id]['gate_type']
        new_state = evaluate_gate(gate, input_states_list)
        states[node_id] = new_state
print("List of states for input vector 1")
for node_id in sorted(g.nodes):
    print(states[node_id],end=" ")
    
print("\n")

count=2
for j in range(2,l+1):
    input_vector=inputs[j].split()    
    for i, input_id in enumerate(primary_inputs):        
        if(states[input_id]!=int(input_vector[i])):   
            states[input_id]=int(input_vector[i])
            for succ_id in g.successors(input_id):
                states[succ_id]=None
                queue.append(succ_id)    
    while queue:
        node_id = queue.pop(0)
        if(states[node_id]==1 or states[node_id]==0):
            continue        
        input_states_list = [states[input_gate_id] for input_gate_id in g.predecessors(node_id)]
        if None in input_states_list:
            queue.append(node_id)
        else:
            gate = g.nodes[node_id]['gate_type']
            new_state = evaluate_gate(gate, input_states_list)
            states[node_id] = new_state
        for succ_id in g.successors(node_id):
            states[succ_id]=None
            queue.append(succ_id)        
    print(f"List of states for input vector {count}")
    for node_id in sorted(g.nodes):
        print(states[node_id],end=" ")
    count+=1
    print("\n")


N1 N2 N22 N23 N3 N6 N7 n_0 n_1 n_2 n_3 

List of states for input vector 1
0 1 1 1 0 0 0 1 1 1 0 

List of states for input vector 2
0 0 0 0 1 0 0 1 1 1 1 

List of states for input vector 3
1 0 0 0 0 0 0 1 1 1 1 

List of states for input vector 4
0 0 0 0 1 1 1 1 0 1 1 

List of states for input vector 5
1 1 1 0 1 1 1 0 0 1 1 

List of states for input vector 6
1 1 1 1 1 0 0 0 1 1 0 

List of states for input vector 7
1 1 1 0 1 1 0 0 0 1 1 

List of states for input vector 8
1 1 1 1 0 0 0 1 1 1 0 

List of states for input vector 9
0 1 1 1 1 0 1 1 1 0 0 

List of states for input vector 10
0 0 0 0 1 1 0 1 0 1 1 



The code reads the circuit and creates DAG and also reads inputs file which contin the primary inputs<br>
To evaluate the logic of each gate,it uses multiple input states<br>
the code then prints the output the gates to the input file<br>
Each input vector is evaluated by changing the states of each primary input, re-evaluating the state of each gate, and then changing the states of the next gates in the circuit in response to the changes.

## Question 3

### Differences between the two above methods

The event driven case is more effetient than the Topological order method<br>
Because in the topological method , first we need to sort the nodes in the topological order and then the evaluation is done in the process where the outputs of the gates are evaluated in a way that whose inputs are available<br>
if the network is large then this process takes much time as it needs tove block by block<br>
In this method many times the same gates are evaluated many times which is time consuming<br> 
But in Event driven case,<br>the code only evaluates the gates only when the inputs are changing, by using priority queue which keeps track of which gate needs to be evaluated next<br>
This method is effecient as we can avoids the need for multiple rounds of evaluation and can be more efficient for large circuits with many gates and inputs<br>
but for the small networks , topological method could be more effecient<br>
because the event-driven simulation approach may require more initial setup time to build the priority queue and initialize the state of the circuit
